<a href="https://colab.research.google.com/github/Stergios-Konstantinidis/CAA/blob/main/functions_and_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import time
import pandas
import random
from datetime import datetime, timedelta

from google.colab import auth
auth.authenticate_user()
print('Authenticated')


from google.cloud import bigquery
# Construct a BigQuery client object.
project_id = 'sbb-project-2023'
client = bigquery.Client(project = project_id)



verboser = True

def verbose(text:str, verboser = verboser):
  if verboser == True:
    print(text)
  else:
    pass


Authenticated


In [2]:
# Set table_id to the ID of the table model to fetch.

def send_beverage_served(user_id, beverage_id):
  table_id = 'projetalcohol.beverage_served'
  to_load = """{"campus_card_idcampus_card":"***","beverage_idbeverage":"****","beverage_serveddate":"*****","beverage_servedtime":"******"}"""
  to_load = to_load.replace("******", str((datetime.now() + timedelta(hours=2)).strftime('%H:%M')+":00.00"))
  to_load = to_load.replace("*****", (datetime.now() + timedelta(hours=2)).strftime('%Y-%m-%d'))
  to_load = to_load.replace("****", str(beverage_id))
  rows_to_insert = [json.loads(to_load.replace("***", user_id))]

  errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
  if errors == []:
    verbose("New rows have been added.")
  else:
    verbose("Encountered errors while inserting rows in beverage served: {}".format(errors))

#send_beverage_served("testuser", 13319)

In [3]:
# Set table_id to the ID of the table model to fetch.
def send_alcohol_level(user_id, alcohol_level):
  table_id = 'projetalcohol.alcohol_level'
  to_load = """{"campus_card_idcampus_card":"***","alcohol_leveldate":"*****","alcohol_leveltime":"******","alcohol_levelethanol":"****"}"""
  to_load = to_load.replace("******", str((datetime.now() + timedelta(hours=2)).strftime('%H:%M:%S')))
  to_load = to_load.replace("*****", (datetime.now() + timedelta(hours=2)).strftime('%Y-%m-%d'))
  to_load = to_load.replace("****", str(alcohol_level))
  rows_to_insert = [json.loads(to_load.replace("***", user_id))]

  errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
  if errors == []:
    verbose("New rows have been added.")
  else:
    verbose("Encountered errors while inserting rows in alcohol level: {}".format(errors))

#send_alcohol_level("testuser", str(7000))

In [5]:
# Set table_id to the ID of the table model to fetch.


def create_user(idcampus_card, campus_cardlastname = "newlastname", campus_cardfirstname = "newfirstname", campus_cardbirth_date = datetime.strptime("1998.10.03", "%Y.%m.%d"), campus_cardabsorption = 4, campus_cardsex = True, campus_cardweight = 70,campus_carddiffusioncoef = 0,alcohol_authorized = 0):
  table_id = 'projetalcohol.campus_card'
  to_load = """{"idcampus_card":"***","campus_cardlastname":"****","campus_cardfirstname":"*****","campus_cardbirth_date":"******","campus_cardabsorption":"*******","campus_cardsex":"********","campus_cardweight":"*********","campus_carddiffusioncoef":"**********","alcohol_limit":"***********"}"""
  to_load = to_load.replace("***********", str(alcohol_authorized))
  to_load = to_load.replace("**********", str(campus_carddiffusioncoef))
  to_load = to_load.replace("*********", str(campus_cardweight))
  to_load = to_load.replace("********", str(campus_cardsex))
  to_load = to_load.replace("*******", str(campus_cardabsorption))
  to_load = to_load.replace("******", str(campus_cardbirth_date.strftime('%Y-%m-%d')))
  to_load = to_load.replace("*****", campus_cardfirstname)
  to_load = to_load.replace("****", campus_cardlastname)
  rows_to_insert = [json.loads(to_load.replace("***", idcampus_card))]

  errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
  if errors == []:
    verbose("New rows have been added.")
  else:
    verbose("Encountered errors while inserting rows in alcohol level: {}".format(errors))

#create_user("UID", "Test", "user", datetime.strptime("1998.10.03", "%Y.%m.%d"),4,True,83,1)


New rows have been added.


In [6]:
def user_exists(campuscard_id):
  q1 = """select campus_card_idcampus_card as campuscard
from sbb-project-2023.projetalcohol.alcohol_level
where campus_card_idcampus_card = "{}"
limit 1""".format(campuscard_id)
  query_job = client.query(q1)
  if len(query_job.to_dataframe()['campuscard'].tolist()) == 0:
    return False
  else:
    return True

user_exists("8843d61d0")

True

In [7]:
# Set table_id to the ID of the table model to fetch.


def create_drink(beverage_name, alcohol_level_percentage, beverage_quantity_cL = 300, beverage_id = random.randrange(2000000)):
  table_id = 'projetalcohol.beverage'
  to_load = """{"idbeverage":"***","beveragename":"****","beveragealcolevel":"*****","beveragebasequantity":"******"}"""
  to_load = to_load.replace("******", str(beverage_quantity_cL))
  to_load = to_load.replace("*****", str(alcohol_level_percentage * beverage_quantity_cL))
  to_load = to_load.replace("****", beverage_name)
  rows_to_insert = [json.loads(to_load.replace("***", str(beverage_id)))]

  errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
  if errors == []:
    verbose("New rows have been added.")
  else:
    verbose("Encountered errors while inserting rows in beverage: {}".format(errors))

#create_drink("Pepsi", 0, 20)

In [8]:
link = "https://api.m5stack.com/v1/D535B855/"
alcohol_level_api = 'https://api.m5stack.com/v1/CAAA456F/call?func=label_1_callback()'
selected_drink = link + "call?func=label_selecteddrink_callback()"
campus_card_uid = link + 'call?func=label_uid_callback()'


drink_recommendation_1 = link + "call?func=input_drink1_callback"
drink_recommendation_2 = link + "call?func=input_drink2_callback"
drink_recommendation_3 = link + "call?func=input_drink3_callback"
reset_selection = link + "call?func=input_selected_drink_reseter_callback"
alcohol_level_setter = link + "call?func=input_Alcohollevel_callback"


In [9]:
def get_data_from_url(url_for_API):
  response = requests.get(url_for_API)
  json_response = json.loads(response.text)
  try:
    text_to_int = int(json_response['data'])
    return str(text_to_int)
  except:
      pass
  return json_response['data'][1:-1]

def send_data_to_url(url_for_API, valueToSend):
  value_to_send_formated = str(url_for_API) + '(%27' + str(valueToSend).replace(' ','%20') + '%27)'
  requests.get(value_to_send_formated)

In [10]:
def get_blood_qtt(userUID):
  q1 = """select campus_cardweight as weight
from sbb-project-2023.projetalcohol.campus_card
where idcampus_card = "{}"
""".format(userUID)
  query_job = client.query(q1)
  user_weight = query_job.to_dataframe()['weight'].to_list()[-1]
  return user_weight * 0.075

get_blood_qtt('82e445122')

6.3

In [18]:
def get_alcohol_per_L_served(userUID):
  querry = """select beverage_served.beverage_serveddate as date, beverage_served.beverage_servedtime as time, (beverage.beveragealcolevel * beverage.beveragebasequantity) as miligrams
  from projetalcohol.beverage_served 
  join projetalcohol.beverage on beverage_served.beverage_idbeverage = beverage.idbeverage
  where beverage_served.campus_card_idcampus_card = '{}'
  order by beverage_served.beverage_serveddate desc, beverage_served.beverage_servedtime desc""".format(userUID)
  query_job = client.query(querry)

  blood_qtt = get_blood_qtt('82e445122')
  dates = query_job.to_dataframe()['date'].to_list()
  times = query_job.to_dataframe()['time'].to_list()
  grams = query_job.to_dataframe()['miligrams'].to_list()

  mgramsPerL = list(map(lambda x: ((x*0.01)/blood_qtt), grams))
  
  values_to_keep_timeage = []
  values_to_keep_grams_PerL = []
  alcohol_level = 0
  for i in range(len(dates)):
    datetimeobject = datetime.strptime(str(dates[i]) + "__" + str(times[i]), "%Y-%m-%d__%H:%M:%S") 
    nowdatetime = datetime.now() + timedelta(hours = 2)

    if datetimeobject + timedelta(hours = 6) > nowdatetime:
      values_to_keep_timeage.append(nowdatetime - datetimeobject)
      values_to_keep_grams_PerL.append(values_to_keep_grams_PerL[i])
    
  if len(values_to_keep_timeage) == 1:
    alcohol_depreciation = ((values_to_keep_timeage[0].total_seconds()/3600) - 1800) * 0.15
    alcohol_level += max(values_to_keep_timeage - alcohol_depreciation, 0)


    
  if len(values_to_keep_timeage) > 1:
    alcohol_depreciation = ((values_to_keep_timeage[0].total_seconds()/3600) - 1800) * 0.15
    alcohol_level += max(values_to_keep_timeage - alcohol_depreciation, 0)
      
  

  return alcohol_level



get_alcohol_per_L_served("8843d61d0")

NameError: ignored

In [ ]:
#dans recommander function, pas besoin
def get_allowed_drinks(alcohol_level, max_level = 0.00025): 
  #todo
  querry = """select * from projetalcohol.beverage where beveragealcolevel * beveragebasequantity <= {} order by beveragealcolevel desc""".format(max_level - alcohol_level)
  query_job = client.query(querry)

  
  return query_job.to_dataframe()

#get_allowed_drinks(0)

,idbeverage,beveragename,beveragealcolevel,beveragebasequantity
0,1888655,Ice Tea,0,20
1,13319,Coke,0,20


In [ ]:
def recommend_drink(user, alcohol_level, max_level = 10000):

  drinks_dataframe = get_allowed_drinks(alcohol_level, max_level)

  #Faire ici la selection des meilleures boissons parmis celles qui figurent dans drinks_dataframe



  
  top_3_finalists = drinks_dataframe['beveragename'].tolist()[0:3]
  indexes = drinks_dataframe['idbeverage'].tolist()[0:3]
  #verbose(top_3_finalists)
  return top_3_finalists[0], top_3_finalists[1], top_3_finalists[2], drinks_dataframe['beveragename'].tolist(), drinks_dataframe['idbeverage'].tolist()

recommend_drink(12,12)

('Vodka',
 'Limoncello',
 'Beer',
 ['Vodka', 'Limoncello', 'Beer', 'Ice Tea', 'Coke'],
 [2938, 93680, 13251, 1888655, 13319])

In [ ]:
def convert_ppm3_to_alcohol_in_blood(user, alcohol_level):
  #todo
  return alcohol_level

In [ ]:
def get_alcohol_level(user_id): #returns the max value of alcohol measured over a period of 9 seconds


  alcohol_level_ppm3 = int(get_data_from_url(alcohol_level_api))
  alcohol_level = convert_ppm3_to_alcohol_in_blood(user_id, alcohol_level_ppm3)

  send_data_to_url(alcohol_level_setter, alcohol_level)
  return alcohol_level

In [ ]:
def on_user_change(): #Function that checks if the campus card placed on the reader has changed => new user
  initial_user = get_data_from_url(campus_card_uid)
  get_alcohol_level(initial_user)
  while True: #repeat this loop until stopped
    send_data_to_url(reset_selection, "None") #ensures no drink is selected (as to avoid the selection being pre-made)
    user_id = get_data_from_url(campus_card_uid) #retrieve the uid of the capuscard located on the reader
    if user_id != initial_user: #checks if our current user has changed
      if user_id != "None" and user_id != "'None'" and user_id != None and user_id != "" and user_id != " " and user_id != "imeou": #ensures the user is not none or an error
        initial_user = user_id #set the initial user to the current user

        if not user_exists(user_id): #checks if the user is in our database
          create_user(user_id) #if no user exists creates the user


        alcohol_level = get_alcohol_level(user_id)
        verbose("Sending alcohol_level user: {}, level: {}".format(user_id, alcohol_level))
        send_alcohol_level(user_id, alcohol_level)


        send_data_to_url(reset_selection, "Non")
        
        max_level = 100000000 #to change
        beverage_1, beverage_2, beverage_3,top3, indexes  = recommend_drink(user_id, alcohol_level, max_level)
        send_data_to_url(drink_recommendation_1, beverage_1)
        send_data_to_url(drink_recommendation_2, beverage_2)
        send_data_to_url(drink_recommendation_3, beverage_3)


        selection = get_data_from_url(selected_drink)
        while selection == "None" or selection == "on":
          time.sleep(1)
          selection = get_data_from_url(selected_drink)

        chosen_drink_index = top3.index(selection)
        drink_id = indexes[chosen_drink_index]
        send_data_to_url(reset_selection, "No")
        verbose("sending_beverage_served user: {}, beverage: {}".format(user_id, selection))
        send_beverage_served(user_id, drink_id)


In [ ]:
on_user_change()

ValueError: ignored